In [1]:
'''
Wrangling npz files:
http://stackoverflow.com/questions/32682928/loading-arrays-from-npz-files-in-pythhon'''

import cv2
import numpy as np
# np.set_printoptions(threshold=np.nan)
import pandas as pd
import matplotlib.pyplot as plt
import glob
from collections import deque

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical
from sklearn.cross_validation import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif, f_regression
from sklearn import preprocessing

/Users/PaulYim/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
Using TensorFlow backend.


In [2]:
# Flip the LEFT & RIGHT directional inputs, but leave FORWARD alone. (for directional target/label data)
# array([[ 1.,  0.,  0.],   <-- FORWARD-LEFT
#        [ 0.,  1.,  0.],   <-- FORWARD-RIGHT
#        [ 0.,  0.,  1.]])  <-- FORWARD

def flip_dir(array):

    array_list = array.tolist()

    final = []
    
    for row in array_list:
        
        # Forward-Left becomes Forward-Right
        if row[0] == 1:
            row = [0,1,0]
            final.append(row)
            
        # Forward-Right becomes Forward-Left
        elif row[1] == 1:
            row = [1,0,0]
            final.append(row)
            
        # Forward remains as Forward
        elif row[2] == 1:
            final.append(row)
            continue
    
    final = np.array(final)       
#     print final
#     print type(final)
    return final

In [3]:
# Unflatten (for image feature data)
def unflatten_flip(array,width,height):
    
    final = []
    
    # Convert the nparray to a list; pull the list from the outer set of square brackets; deque the list.
#     array_list = array.tolist()

#     print 'This is ARRAY: ',array

#     for a in array:
#         a = list(a)
#         print 'THIS IS a: ', a
#         initial = deque(a)
    
    
    initial = deque(array)
    
    # For each row in the specified height dimension...
    for row in xrange(height):
        temp_row = []

        # ...for each column in the specified width dimension...
        for col in xrange(width):
            
            # ...pop off the last item in the initial array and append to the new temp_row.
            d = initial.pop()
            temp_row.append(d)
            
        # Append temp_row to final, and reset temp_row to empty.
        final.append(temp_row)
        temp_row = []

    # Now we have the final list, but it's 'upside down' because we took care of left to right, but not bottom to top.
    final = deque(final)
    
    # This takes care of bottom to top.
    final.reverse()
   
    final = np.array(final)       
#     print final
#     print type(final)
    return final

In [8]:
# load training data
image_array = np.zeros((1, 38400))
label_array = np.zeros((1, 3), 'float')
training_data = glob.glob('training_data/testRT4.npz')        # Finds filenames matching specified path or pattern.

for single_npz in training_data:                        
    with np.load(single_npz) as data:
#         print data.keys()
        
        # LABELS (this works)        
        labels = data.f.train_labels
        print 'Original Labels dims: ', labels.shape
        
        labels_flipped = flip_dir(labels)
        
        # FEATURES (this works)
        image_array = np.zeros((1, 38400))
        
        original_features = data.f.train
        print 'Original Features dims: ', original_features.shape
        
        for feature in original_features:
            
#             print 'This is the ORIGINAL feature: ', type(feature), feature.shape, feature
            feature = list(feature)
            
            feature_array_flipped = unflatten_flip(feature,width=320,height=120)
            
            temp_array = feature_array_flipped.reshape(1, 38400).astype(np.float32)
#             print 'This is the FLIPPED feature: ', type(temp_array), temp_array.shape, temp_array 
            
            image_array = np.vstack((image_array, temp_array))
            
        train_labels = labels_flipped        
        train = image_array[1:, :]

        print 'Flipped Labels dims: ', train_labels.shape
        print 'Flipped Features dims: ', train.shape        
        
        np.savez('training_data_temp/testFLIPPED.npz', train=train, train_labels=train_labels)      
        
        print 'Flipping is complete.'

Original Labels dims:  (184, 3)
Original Features dims:  (184, 38400)
Flipped Labels dims:  (184, 3)
Flipped Features dims:  (184, 38400)
Flipping is complete.
